# 📓 The GenAI Revolution Cookbook

**Title:** How to Build CrewAI Tools with Decorators, BaseTool, and Payloads

**Description:** Choose the right CrewAI tools—decorators, BaseTool, or payload—validate structured inputs with Pydantic, and ship reliable agent integrations faster with confidence.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



CrewAI agents need tools to interact with external systems, validate data, and execute business logic. Without tools, agents can only generate text. With tools, they can log events, call APIs, enforce schemas, and integrate with your production stack.

This guide shows you three patterns for building custom CrewAI tools: a simple function decorator for quick prototyping, a BaseTool subclass with Pydantic validation for production-grade type safety, and a payload-based tool for flexible, dynamic schemas. You'll finish with a fully runnable Colab notebook that demonstrates all three patterns end-to-end, ready to copy, paste, and execute.

## Why Use CrewAI for This Problem

CrewAI's tool system is designed for agent-driven workflows where you need automatic tool discovery, schema enforcement, and iterative development. Compared to alternatives:

- **OpenAI function calling**: Requires manual JSON Schema definitions and separate validation logic. CrewAI integrates Pydantic schemas directly into tools, reducing boilerplate.
- **LangChain Tools**: Similar decorator pattern, but CrewAI's BaseTool subclass offers tighter integration with agent state and crew orchestration, making it easier to manage multi-agent workflows.
- **Direct JSON Schema interfaces**: More flexible but error-prone. CrewAI enforces schemas at the tool level, catching errors before they reach your business logic.

For builders who want to move fast, validate inputs automatically, and scale to multi-agent systems, CrewAI's tool patterns offer a practical balance of simplicity and robustness.

## Core Concepts for This Use Case

**Tool**: A function or class that an agent can call. Tools are registered with agents and discovered automatically during task execution.

**@tool decorator**: Wraps a Python function into a CrewAI tool. Best for simple, stateless operations with basic type hints.

**BaseTool subclass**: A class-based tool with Pydantic schema validation via `args_schema`. Use this for production workflows where you need strict input validation, internal state, or business rule enforcement.

**Payload tool**: A tool that accepts a JSON string payload. Useful when input schemas vary across contexts or when you need to pass complex, nested data structures.

**args_schema**: A Pydantic `BaseModel` that defines the expected input structure for a BaseTool. CrewAI validates inputs against this schema before calling the tool's `_run` method.

## Setup

Install the required dependencies. We use `crewai` for the agent framework, `crewai-tools` for stable tool imports, and `pydantic` for schema validation.

In [ ]:
!pip install -q crewai crewai-tools pydantic~=2.8

Import the core classes and set up your API key. If running in Colab, use the userdata API to load secrets securely. Otherwise, use a runtime prompt fallback.

In [ ]:
import os
import json
from typing import Type
from getpass import getpass
from crewai import Agent, Task, Crew
from crewai_tools import tool, BaseTool
from pydantic import BaseModel, Field, ValidationError

# Load API key from environment or prompt at runtime
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")

If you're running in Google Colab and have stored your API key in Colab Secrets, use this block instead to load it automatically.

In [ ]:
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError

keys = ["OPENAI_API_KEY"]
missing = []
for k in keys:
    value = None
    try:
        value = userdata.get(k)
    except SecretNotFoundError:
        pass
    os.environ[k] = value if value is not None else ""
    if not os.environ[k]:
        missing.append(k)

if missing:
    raise EnvironmentError(f"Missing keys: {', '.join(missing)}. Add them in Colab Settings, Secrets.")

print("All keys loaded.")

## Using the Tool in Practice

### Approach 1: Simple Function Tool with @tool Decorator

The `@tool` decorator is the fastest way to turn a Python function into a CrewAI tool. Use it for quick prototypes or stateless operations where you don't need complex validation.

Define a function that logs a user intent. The function signature becomes the tool's input schema, and the docstring becomes the tool's description.

In [ ]:
@tool("user_intent_logger")
def user_intent_logger(user_name: str, intent: str, is_premium_user: bool) -> str:
    """
    Log a user intent with basic parameters.
    
    Args:
        user_name (str): The display name of the user.
        intent (str): The action the user wants to perform.
        is_premium_user (bool): Whether the user is on a premium plan.
    
    Returns:
        str: JSON string with status and a synthetic log_id.
    """
    log_event = {
        "user_name": user_name,
        "intent": intent,
        "is_premium_user": is_premium_user,
    }
    print(f"[user_intent_logger] Logged event: {log_event}")
    response = {"status": "ok", "log_id": f"{user_name.lower()}-{intent}-001"}
    return json.dumps(response)

Create an agent that uses the tool. The agent will automatically discover the tool and call it when the task requires logging.

In [ ]:
simple_agent = Agent(
    role="Support Analyst",
    goal="Understand user intent and log it using provided tools",
    backstory="You review user requests and ensure their intent is captured for analytics",
    tools=[user_intent_logger],
    model="gpt-4o-mini",
    verbose=True,
)

Define a task that instructs the agent to call the logger tool with explicit parameters.

In [ ]:
simple_task = Task(
    description=(
        "A user named Alice wants to cancel her plan. "
        "Call the user_intent_logger tool with user_name='Alice', intent='cancel', is_premium_user=True. "
        "Return only the JSON from the tool output."
    ),
    expected_output="A JSON string with status and log_id fields.",
    agent=simple_agent,
)

Run the crew to execute the task. The agent will call the tool and return the result.

In [ ]:
simple_crew = Crew(
    agents=[simple_agent],
    tasks=[simple_task],
    verbose=True,
)

print("=== Running Approach 1: Decorator tool ===")
result_1 = simple_crew.kickoff()
print("Crew result:", result_1)

### Approach 2: BaseTool Subclass with Pydantic Validation

For production workflows, use a `BaseTool` subclass with a Pydantic schema. This enforces strict input validation, supports internal state, and lets you add business rules.

Define a Pydantic schema for the tool's inputs. This schema will be validated automatically before the tool runs.

In [ ]:
class IntentInput(BaseModel):
    """
    Pydantic schema for user intent logging.
    
    Args:
        user_name (str): End user display name (min 1 character).
        intent (str): Action the user wants to perform (min 2 characters).
        is_premium_user (bool): True if the user is on a premium plan.
    """
    user_name: str = Field(..., min_length=1, description="End user display name")
    intent: str = Field(..., min_length=2, description="Action the user wants to perform")
    is_premium_user: bool = Field(..., description="True if the user is on a premium plan")

Implement a `BaseTool` subclass that uses the schema. The `_run` method contains your business logic and is called only after validation succeeds.

In [ ]:
class UserIntentLoggerTool(BaseTool):
    """
    CrewAI BaseTool for validated user intent logging.
    
    Attributes:
        name (str): Tool name for agent discovery.
        description (str): Tool description for agent context.
        args_schema (Type[BaseModel]): Pydantic schema for input validation.
    """
    name: str = "user_intent_logger_validated"
    description: str = (
        "Logs validated user intents to the analytics pipeline. "
        "Requires user_name, intent, and is_premium_user."
    )
    args_schema: Type[BaseModel] = IntentInput

    def __init__(self):
        super().__init__()
        self._log_counter = 0

    def _run(self, user_name: str, intent: str, is_premium_user: bool) -> str:
        """
        Log a validated user intent with business rule enforcement.
        
        Args:
            user_name (str): The display name of the user.
            intent (str): The action the user wants to perform.
            is_premium_user (bool): Whether the user is on a premium plan.
        
        Returns:
            str: JSON string with status and log_id.
        
        Raises:
            ValueError: If intent is not in the allowed set.
        """
        if intent.lower() not in {"cancel", "upgrade", "downgrade", "support"}:
            raise ValueError("intent must be one of cancel, upgrade, downgrade, support")
        self._log_counter += 1
        log_id = f"{user_name.lower()}-{intent}-{self._log_counter:03d}"
        event = {
            "user_name": user_name,
            "intent": intent.lower(),
            "is_premium_user": is_premium_user,
            "log_id": log_id,
        }
        print(f"[user_intent_logger_validated] Logged event: {event}")
        return json.dumps({"status": "ok", "log_id": log_id})

Instantiate the tool and wire it into an agent. The agent will use the validated tool for all logging tasks.

In [ ]:
validated_tool = UserIntentLoggerTool()

validated_agent = Agent(
    role="Support Analyst",
    goal="Log validated user intents with strict schema and business rules",
    backstory="You ensure data quality by using validated tools",
    tools=[validated_tool],
    model="gpt-4o-mini",
    verbose=True,
)

validated_task = Task(
    description=(
        "A user named Bob wants to upgrade his plan. "
        "Call the user_intent_logger_validated tool with user_name='Bob', intent='upgrade', is_premium_user=False. "
        "Return only the JSON from the tool output."
    ),
    expected_output="A JSON string with status and log_id fields.",
    agent=validated_agent,
)

validated_crew = Crew(
    agents=[validated_agent],
    tasks=[validated_task],
    verbose=True,
)

print("=== Running Approach 2: BaseTool with Pydantic ===")
result_2 = validated_crew.kickoff()
print("Crew result:", result_2)

Test validation failure by passing an invalid intent. The tool will raise a `ValueError` before executing any business logic.

In [ ]:
invalid_task = Task(
    description=(
        "A user named Carol wants a refund immediately. "
        "Call the user_intent_logger_validated tool with user_name='Carol', intent='refund', is_premium_user=True. "
        "Return only the JSON from the tool output."
    ),
    expected_output="A JSON string with status and log_id fields.",
    agent=validated_agent,
)

invalid_crew = Crew(
    agents=[validated_agent],
    tasks=[invalid_task],
    verbose=True,
)

print("=== Running Approach 2b: Expect validation failure ===")
try:
    invalid_crew.kickoff()
except Exception as e:
    print("Caught error as expected:", repr(e))

### Approach 3: Payload-Based Tool for Flexible Schemas

When input schemas vary across contexts or you need to pass complex, nested data, use a payload-based tool that accepts a JSON string.

Define a tool that parses and validates a JSON payload. This tool checks for required keys and types, then logs the event.

In [ ]:
@tool("payload_intent_logger")
def payload_intent_logger(payload: str) -> str:
    """
    Log a user intent from a JSON string payload.
    
    Args:
        payload (str): JSON string with required keys user_name (str), intent (str).
            Optional keys: is_premium_user (bool), timestamp (str ISO), context (dict).
    
    Returns:
        str: JSON string with status and log_id.
    
    Raises:
        ValueError: If payload is not valid JSON or required keys are missing/invalid.
    """
    try:
        data = json.loads(payload)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON payload. Error: {e}")

    for key in ["user_name", "intent"]:
        if key not in data or not isinstance(data[key], str) or not data[key]:
            raise ValueError(f"Missing or invalid required key: {key}")

    if "is_premium_user" in data and not isinstance(data["is_premium_user"], bool):
        raise ValueError("is_premium_user must be a boolean")

    user_name = data["user_name"]
    intent = data["intent"].lower()
    is_premium_user = data.get("is_premium_user", False)
    log_id = f"{user_name.lower()}-{intent}-p1"

    print(f"[payload_intent_logger] Logged payload: {data}")
    return json.dumps({"status": "ok", "log_id": log_id})

Create an agent and task for the payload-based tool. The agent will construct a JSON string and pass it to the tool.

In [ ]:
payload_agent = Agent(
    role="Support Analyst",
    goal="Construct valid JSON payloads and log them via payload_intent_logger",
    backstory="You prefer flexible payloads when fields vary across contexts",
    tools=[payload_intent_logger],
    model="gpt-4o-mini",
    verbose=True,
)

payload_task = Task(
    description=(
        "Construct a strict JSON string with required keys user_name and intent. "
        "Optional key is_premium_user may be included as a boolean. "
        "Then call payload_intent_logger(payload=<the JSON string>) for user_name='Dana', intent='downgrade', is_premium_user=false. "
        "Return only the JSON from the tool output."
    ),
    expected_output="A JSON string with status and log_id fields.",
    agent=payload_agent,
)

payload_crew = Crew(
    agents=[payload_agent],
    tasks=[payload_task],
    verbose=True,
)

print("=== Running Approach 3: Payload tool ===")
result_3 = payload_crew.kickoff()
print("Crew result:", result_3)

## Run and Evaluate

Validate that all three approaches produce the expected JSON output with `status` and `log_id` fields.

In [ ]:
def ensure_json_ok(output: str) -> bool:
    """
    Assert that the output is valid JSON with required fields.
    
    Args:
        output (str): Output string from a tool.
    
    Returns:
        bool: True if output is valid and contains required fields.
    
    Raises:
        AssertionError: If output is not valid JSON or missing required fields.
    """
    try:
        data = json.loads(output)
    except Exception as e:
        raise AssertionError(f"Output is not valid JSON. Error: {e}")
    assert "status" in data and data["status"] == "ok", "Missing or bad status"
    assert "log_id" in data, "Missing log_id"
    return True

print("=== Validating Approach 1 output ===")
assert ensure_json_ok(result_1), "Approach 1 output invalid"

print("=== Validating Approach 2 output ===")
assert ensure_json_ok(result_2), "Approach 2 output invalid"

print("=== Validating Approach 3 output ===")
assert ensure_json_ok(result_3), "Approach 3 output invalid"

print("All validations passed.")

Test edge cases to confirm validation behavior. For the BaseTool, pass an empty `user_name` to trigger a Pydantic validation error.

In [ ]:
edge_task = Task(
    description=(
        "Call the user_intent_logger_validated tool with user_name='', intent='support', is_premium_user=True. "
        "Return the tool output."
    ),
    expected_output="A JSON string with status and log_id fields.",
    agent=validated_agent,
)
edge_crew = Crew(agents=[validated_agent], tasks=[edge_task], verbose=True)

print("=== BaseTool invalid input: Expect Pydantic error ===")
try:
    edge_crew.kickoff()
except Exception as e:
    print("Caught error as expected:", repr(e))

Test the payload tool with malformed JSON to confirm error handling.

In [ ]:
bad_payload_task = Task(
    description=(
        "Call payload_intent_logger with payload set to this invalid JSON string exactly: "
        "{user_name: Dana, intent: downgrade} "
        "Return the tool output."
    ),
    expected_output="Tool raises ValueError due to invalid JSON.",
    agent=payload_agent,
)

bad_payload_crew = Crew(
    agents=[payload_agent],
    tasks=[bad_payload_task],
    verbose=True,
)

print("=== Payload tool invalid JSON: Expect failure ===")
try:
    bad_payload_crew.kickoff()
except Exception as e:
    print("Caught error as expected:", repr(e))

## Conclusion

You now have three production-ready patterns for building custom CrewAI tools. Use the `@tool` decorator for quick prototypes, `BaseTool` with Pydantic for strict validation and business rules, and payload-based tools for flexible, dynamic schemas. All three patterns are runnable in Colab and ready to integrate into your agent workflows.

Next steps: Extend the BaseTool pattern to call external APIs, add structured logging with timestamps and trace IDs, or combine multiple tools in a single crew to build multi-step workflows. For advanced validation, explore Pydantic's custom validators and field constraints to enforce domain-specific rules at the schema level.